In [3]:
!pip install pydub pytube

from google.colab import drive
from pydub import AudioSegment
from pytube import YouTube
from concurrent.futures import ThreadPoolExecutor
import csv
import os

# Montowanie Google Drive
drive.mount('/content/drive')

# Ścieżki do folderów i plików na Google Drive
folder_nadrzedny = '/content/drive/My Drive/Statystyka Wielowymiarowa - Zadanie 2'
folder_audio = os.path.join(folder_nadrzedny, 'piosenki - audio')
folder_oryginalne = os.path.join(folder_nadrzedny, 'oryginalne piosenki')
plik_csv = os.path.join(folder_nadrzedny, 'etykiety piosenek.csv')

# Tworzenie folderów, jeśli nie istnieją
if not os.path.exists(folder_nadrzedny):
    os.makedirs(folder_nadrzedny)
if not os.path.exists(folder_audio):
    os.makedirs(folder_audio)
if not os.path.exists(folder_oryginalne):
    os.makedirs(folder_oryginalne)

# Funkcja do pobierania i skracania piosenek
def pobierz_i_skroc(url, gatunek, format='wav'):
    yt = YouTube(url)
    stream = yt.streams.filter(only_audio=True).first()
    downloaded_file = stream.download(folder_oryginalne)
    audio = AudioSegment.from_file(downloaded_file)

    # Długość piosenki w milisekundach
    dlugosc_piosenki = len(audio)

    # Obliczenie liczby fragmentów (maksymalnie 5)
    liczba_fragmentow = min(5, dlugosc_piosenki // 30000)

    for i in range(liczba_fragmentow):
        start_fragmentu = i * 30000
        koniec_fragmentu = start_fragmentu + 30000
        skrocone_audio = audio[start_fragmentu:koniec_fragmentu]
        skrocony_plik = os.path.join(folder_audio, yt.title + f'_fragment_{i+1}.{format}')
        skrocone_audio.export(skrocony_plik, format=format)
        with open(plik_csv, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([skrocony_plik, gatunek])
        print(f'Fragment {i+1} piosenki został zapisany: {skrocony_plik}')

    print(f'Pomyślnie zapisano {liczba_fragmentow} fragmenty/ów piosenki: {url}')

# Gatunki muzyczne do wyboru
gatunki = ['rock', 'pop', 'hiphop', 'klasyczna', 'metal']

# Funkcja dodająca nową piosenkę
def nowa_piosenka(url, gatunek):
    pobierz_i_skroc(url, gatunek)

# Tworzenie pliku CSV, jeśli nie istnieje
if not os.path.exists(plik_csv):
    with open(plik_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Sciezka', 'Gatunek'])

# Uruchomienie wielowątkowości dla pobierania i przetwarzania piosenek
with ThreadPoolExecutor(max_workers=5) as executor:
    while True:
        url = input("Podaj URL piosenki z YouTube: ")
        print("Wybierz gatunek muzyki:")
        for i, gatunek in enumerate(gatunki, start=1):
            print(f"{i}. {gatunek}")
        wybor = int(input("Numer wybranego gatunku: "))
        gatunek = gatunki[wybor - 1]

        # Dodawanie zadania do puli wątków
        executor.submit(nowa_piosenka, url, gatunek)

        if input("Czy chcesz dodać kolejną piosenkę? (tak/nie): ").lower() != 'tak':
            break

print("Proces dodawania piosenek zakończony.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Podaj URL piosenki z YouTube: https://youtu.be/X4bgXH3sJ2Q?feature=shared
Wybierz gatunek muzyki:
1. rock
2. pop
3. hiphop
4. klasyczna
5. metal
Numer wybranego gatunku: 5
Czy chcesz dodać kolejną piosenkę? (tak/nie): tak
Fragment 1 piosenki został zapisany: /content/drive/My Drive/Statystyka Wielowymiarowa - Zadanie 2/piosenki - audio/The Trooper_fragment_1.wav
Fragment 2 piosenki został zapisany: /content/drive/My Drive/Statystyka Wielowymiarowa - Zadanie 2/piosenki - audio/The Trooper_fragment_2.wav
Fragment 3 piosenki został zapisany: /content/drive/My Drive/Statystyka Wielowymiarowa - Zadanie 2/piosenki - audio/The Trooper_fragment_3.wav
Fragment 4 piosenki został zapisany: /content/drive/My Drive/Statystyka Wielowymiarowa - Zadanie 2/piosenki - audio/The Trooper_fragment_4.wav
Fragment 5 piosenki został zapisany: /content/drive/My Drive/Statystyka Wielo